### Automate tickets remainder process

The main goal of this python code is to automate the process of sending emails to employees that have tickets open for more than 3 days. 
This task has been performed with excel VBA but python can do the exact same thing in 3 minutes. In the past it would take me about 3 hours per week to send out the remainder emails to everyone on the team.

In [5]:
# import library needed

import pandas as pd
import smtplib
import datetime
import os
import re

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
# open document
data = pd.read_excel('yourfilename.xlsx')
data.head()

In [6]:
# dimension of data
data.shape

(208, 12)

In [18]:
# people assigned with tickets from sso
df = data[(data['Assignment group']=='sso-helpdesk-hr') | (data['Assignment group']=='sso-helpdesk-p2p') 
     | (data['Assignment group']=='sso-management')| (data['Assignment group']=='sso-p2p-ach-sua')]

In [9]:
df.shape

(174, 12)

In [10]:
# divide the fin tickets in personal files
for name, g in df.groupby('Assigned to'):
    g.to_excel(f'{name}.xlsx', index=False)

In [11]:
# get the path where all files arfe stored
path = 'Users/yourname/Documents/E-ticketing/20191015'
path = os.getcwd()
files = os.listdir(path)

In [13]:
# selected xlsx as they correspond to ticket's files
files_xlsx = [f for f in files if f[-6:] == ').xlsx']


In [ ]:
# get list of names
str1 = ''.join(files_xlsx)
index = re.findall(r'\((.*?)\)', str1)
print(index) 

# get list of names and add university email
string = '@uchicago.edu'
emails_list = [x + string for x in index]


In [15]:
# dictionary with managers and employees that will recieve email with tickets open
# people - managers
managers = {'teammember@uchicago.edu':'manager@uchicago.edu'            
           }

In [16]:
def send_email_work(f):
    user = re.findall(r'\((.*?)\)', f)[0]
    end_email = '@uchicago.edu'
    single_email = user + end_email
    print(single_email)    
    
    cc_email = []
    if single_email in managers:
        cc_email.append(managers[single_email])

    # email details 
    email_user = 'youremail@uchicago.edu'
    email_password = 'yourpassword'
    email_send = single_email
    subject = 'yoursubject'
    cc = cc_email
    
    # compose msg
    msg = MIMEMultipart()
    msg['From'] = email_user
    msg['To'] = email_send
    msg['Subject'] = subject
    msg['CC'] = ','.join(cc)
    date = datetime.date.today().strftime('%m-%d-%Y')


    # msg with attachment file
    body = 'Hello, \n\nThis is courtesy email regarding ServiceNow tickets. Attached are SN tickets assigned to you that, as of '+ date + ', have not been updated in three or more days.'
    msg.attach(MIMEText(body, 'plain'))
    filename = f
    attachment = open(filename, 'rb')

    # set a part to read the email and attachated doc
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+filename)
    msg.attach(part)

    # text of the email
    text = msg.as_string()

    # connect to server
    email_send = [single_email] + cc
    
    #email_send = email_send
    server = smtplib.SMTP('smtp-mail.outlook.com', 587)
    server.starttls()
    server.login(email_user,email_password)

    # send and close application
    server.sendmail(email_user,email_send,text)
    server.quit()
    return


In [ ]:
for f in files:
    if f[-6:] == ').xlsx': 
        if f[0].isupper():
            send_email_work(f)
            